<img src="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png" srcset="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_130 130w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_260 260w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_390 390w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_520 520w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_650 650w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_780 780w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_910 910w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1040 1040w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1170 1170w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1290 1290w" sizes="100vw" width="1290">

<p style='margin-top: 1rem; margin-bottom: 1rem;'>Developed by Sebastian Gingter, Developer Consultant @ <a href='https://thinktecture.com' _target='blank'>Thinktecture AG</a> -- More about me on my <a href='https://thinktecture.com/sebastian-gingter' _target='blank'>profile page</a></p>

# Tool Calling with LLMs

This notebook demonstrates how to implement and use tool/function calling.

Note: This notebook uses OpenAI directly, as the models on OVH currently do not support tool / function calling.

## Prettify Colab Notebook outputs

In [1]:
from IPython.display import HTML, display

def set_css():
    display(HTML('''
    <style>
        pre {
            white-space: pre-wrap;
        }
    </style>
    '''))

get_ipython().events.register('pre_run_cell', set_css)

## Set API keys via Colab Secrets

In [6]:
from google.colab import userdata
import os

# set LLM Api Key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

## Install dependencies

In [2]:
# Load libraries
!pip -q install langchain==0.3.7 langchain-openai==0.2.5 langchain-community==0.3.5

# Check versions
!pip show langchain
!pip show langchain_openai
!pip show langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
Name: langchain
Version: 0.3.7
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
Name: langchain-openai
Version: 0.2.5
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /

## Prepare LLM

In [7]:
import random
from typing import Dict, Any
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

MODEL_NAME = 'gpt-4o'

TEMPERATURE = 0
MAX_TOKENS = 1500

llm = ChatOpenAI(
    model=MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    base_url=OPENROUTER_BASE_URL
)

## Prepare Tools

In [4]:
# Function 1: Get user ID from name
def get_user_id(name: str) -> Dict[str, Any]:
    """
    Returns a random user ID for a given name.

    Args:
        name (str): The name of the user

    Returns:
        dict: A dictionary containing the user_id and name
    """
    # In a real application, this would query a database
    user_id = random.randint(1000, 9999)
    return {
        "user_id": user_id,
        "name": name
    }

# Function 2: Book meeting room
def book_meeting_room(user_id: int, start_date: str, end_date: str) -> Dict[str, Any]:
    """
    Books a meeting room for a given time period.

    Args:
        user_id (int): The ID of the user booking the room
        start_date (str): Start date and time (format: YYYY-MM-DD HH:MM)
        end_date (str): End date and time (format: YYYY-MM-DD HH:MM)

    Returns:
        dict: A dictionary containing the booking details
    """
    # In a real application, this would interact with a booking system
    return {
        "status": "success",
        "message": f"Meeting room booked for user {user_id} from {start_date} to {end_date}",
        "booking_id": random.randint(10000, 99999)
    }

## Configure Tools for LLM

In [8]:
# Define the tools for LangChain
from langchain.tools import StructuredTool

# Create structured tools
tools = [
    StructuredTool.from_function(
        func=get_user_id,
        name="get_user_id",
        description="Retrieves a user's ID based on the name (or part of the name, even initials work) of the user."
    ),
    StructuredTool.from_function(
        func=book_meeting_room,
        name="book_meeting_room",
        description="Book a meeting room for a specific time period"
    )
]

# Create prompt template with agent_scratchpad
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools to help fulfil your user's request. Think about a solution, call tools if need be and then answer."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create agent
agent = create_openai_tools_agent(llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Test the function calling system

In [9]:

# Test 1: Getting a user ID
print("Test 1: Getting a user ID")
result = agent_executor.invoke({"input": "Can you get me the user ID for John Smith?"})
print("Result:", result["output"])
print("\n" + "-"*50 + "\n")

# Test 2: Booking a meeting room
print("Test 2: Booking a meeting room")
result = agent_executor.invoke({"input": "I need to book a meeting room for user John Smith tomorrow from 2 PM to 3 PM"})
print("Result:", result["output"])

Test 1: Getting a user ID


> Entering new AgentExecutor chain...

Invoking: `get_user_id` with `{'name': 'John Smith'}`


{'user_id': 6597, 'name': 'John Smith'}The user ID for John Smith is 6597.

> Finished chain.
Result: The user ID for John Smith is 6597.

--------------------------------------------------

Test 2: Booking a meeting room


> Entering new AgentExecutor chain...

Invoking: `get_user_id` with `{'name': 'John Smith'}`


{'user_id': 7777, 'name': 'John Smith'}
Invoking: `book_meeting_room` with `{'user_id': 7777, 'start_date': '2023-11-30T14:00:00', 'end_date': '2023-11-30T15:00:00'}`


{'status': 'success', 'message': 'Meeting room booked for user 7777 from 2023-11-30T14:00:00 to 2023-11-30T15:00:00', 'booking_id': 43471}The meeting room has been successfully booked for John Smith on November 30th, 2023, from 2 PM to 3 PM. The booking ID is 43471.

> Finished chain.
Result: The meeting room has been successfully booked for John Smith on November 30th, 2023, from 2 PM 

## Alternative usage with annotations

In [11]:
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.agents.agent import RunnableMultiActionAgent

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    print(f"Multiplying {first_int} and {second_int}")
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    """Add two integers."""
    print(f"Adding {first_int} and {second_int}")
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    """Exponentiate the base to the exponent power."""
    print(f"Exponentiating {base} to the {exponent} power")
    return base**exponent

tools = [multiply, add, exponentiate]


prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant."),
  ("placeholder", "{chat_history}"),
  ("human", "{input}"),
  ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
if isinstance(agent_executor.agent, RunnableMultiActionAgent):
    agent_executor.agent.stream_runnable = False

agent_executor.invoke({
    "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


Exponentiating 3 to the 5 power
243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


Adding 12 and 3
15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


Multiplying 243 and 15
3645
Invoking: `exponentiate` with `{'base': 3645, 'exponent': 2}`


Exponentiating 3645 to the 2 power
13286025The result of taking 3 to the fifth power, multiplying it by the sum of twelve and three, and then squaring the whole result is 13,286,025.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of taking 3 to the fifth power, multiplying it by the sum of twelve and three, and then squaring the whole result is 13,286,025.'}